In [5]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from scipy.interpolate import griddata
import regionmask
import glob
import os

In [6]:
#Opening and selecting the CF data
path = '/home/rcostell/Observation_Files/CALIPSO_GOCCP/LowMidHigh/'

file_pattern = '*.nc'

files = sorted(glob.glob(os.path.join(path, file_pattern)))

f = xr.open_mfdataset(files, combine='nested', concat_dim='time', decode_cf=True)

#Resample the Cloud Fraction times
f['time'] = xr.decode_cf(f)['time']
f = f.resample(time='1M').mean()

lat0 = 60
lat1 = 90
f = f.rename({'latitude':'lat'}).rename({'longitude':'lon'})
f = f.sel(lat = slice(lat0, lat1))
lon_plot = f.lon.values
lat_plot = f.lat.values
f

/home/rcostell/.conda/envs/met2025/lib/python3.10/site-packages/xarray/groupers.py:490: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(


<xarray.Dataset> Size: 21MB
Dimensions:     (time: 96, lat: 30, lon: 360, toplvl: 3)
Coordinates:
  * lat         (lat) float32 120B 60.5 61.5 62.5 63.5 ... 86.5 87.5 88.5 89.5
  * lon         (lon) float32 1kB -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * toplvl      (toplvl) float32 12B 0.0 0.0 0.0
  * time        (time) datetime64[ns] 768B 2007-01-31 2007-02-28 ... 2014-12-31
Data variables:
    clccalipso  (time, lat, lon) float32 4MB dask.array<chunksize=(1, 30, 360), meta=np.ndarray>
    clhcalipso  (time, lat, lon) float32 4MB dask.array<chunksize=(1, 30, 360), meta=np.ndarray>
    cllcalipso  (time, lat, lon) float32 4MB dask.array<chunksize=(1, 30, 360), meta=np.ndarray>
    clmcalipso  (time, lat, lon) float32 4MB dask.array<chunksize=(1, 30, 360), meta=np.ndarray>
    cltcalipso  (time, lat, lon) float32 4MB dask.array<chunksize=(1, 30, 360), meta=np.ndarray>
Attributes:
    Description:               GOCCP_Map_Low_Mid_High_file
    Date:                      
    Version:                   Prog_version
    Author:                    Gregory CESANA, Helene CHEPFER, LMD/IPSL
    Scientific_contact:        helene.chepfer@lmd.polytechnique.fr
    Technical_support:         gregory.cesana@lmd.polytechnique.fr
    Creationdate:              20190201
    Website:                   http://climserv.ipsl.polytechnique.fr/cfmip-ob...
    References:                Chepfer et al, 2010, The GCM Oriented CALIPSO ...
    history:                   Wed Oct  9 15:42:51 2019: ncks -O -4 -L 1 MapL...
    NCO:                       netCDF Operators version 4.7.9 (Homepage = htt...
    nco_openmp_thread_number:  1

In [7]:
path = '/home/rcostell/Observation_Files/NSIDC/Regridded_Sea_Ice_Concentration/Monthly/Northern_Hemisphere/'
# Define the pattern to match the files
file_pattern = '*_nh_*regridded*.nc'

# Find all files matching the pattern
files = sorted(glob.glob(path + file_pattern))

f0 = xr.open_mfdataset(files, combine='nested', concat_dim='tdim')

f0 = f0.rename({'tdim': 'time'})
f0['time'] = xr.decode_cf(f0)['time']
f0 = f0.interp(lat=lat_plot, lon=lon_plot)

ind = []
valid_times = []

tolerance = np.timedelta64(1, 's')
f0 = f0.resample(time='1M').mean()
f0.time

ind = []
for i, d in enumerate(f.time.values):
    tmp = np.where(f0.time.values == d)[0]
    ind.append(tmp[0])

ind = np.array(ind)

f0 = f0.isel(time = ind).cdr_seaice_conc_gridded
f0['time'] = f.time
f = f.assign(seaice_conc=f0)
f

/tmp/ipykernel_378138/1124953053.py:10: UserWarning: rename 'tdim' to 'time' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  f0 = f0.rename({'tdim': 'time'})
/home/rcostell/.conda/envs/met2025/lib/python3.10/site-packages/xarray/groupers.py:490: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(


<xarray.Dataset> Size: 29MB
Dimensions:      (time: 96, lat: 30, lon: 360, toplvl: 3)
Coordinates:
  * lat          (lat) float32 120B 60.5 61.5 62.5 63.5 ... 86.5 87.5 88.5 89.5
  * lon          (lon) float32 1kB -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * toplvl       (toplvl) float32 12B 0.0 0.0 0.0
  * time         (time) datetime64[ns] 768B 2007-01-31 2007-02-28 ... 2014-12-31
Data variables:
    clccalipso   (time, lat, lon) float32 4MB dask.array<chunksize=(1, 30, 360), meta=np.ndarray>
    clhcalipso   (time, lat, lon) float32 4MB dask.array<chunksize=(1, 30, 360), meta=np.ndarray>
    cllcalipso   (time, lat, lon) float32 4MB dask.array<chunksize=(1, 30, 360), meta=np.ndarray>
    clmcalipso   (time, lat, lon) float32 4MB dask.array<chunksize=(1, 30, 360), meta=np.ndarray>
    cltcalipso   (time, lat, lon) float32 4MB dask.array<chunksize=(1, 30, 360), meta=np.ndarray>
    seaice_conc  (time, lat, lon) float64 8MB dask.array<chunksize=(1, 30, 360), meta=np.ndarray>
Attributes:
    Description:               GOCCP_Map_Low_Mid_High_file
    Date:                      
    Version:                   Prog_version
    Author:                    Gregory CESANA, Helene CHEPFER, LMD/IPSL
    Scientific_contact:        helene.chepfer@lmd.polytechnique.fr
    Technical_support:         gregory.cesana@lmd.polytechnique.fr
    Creationdate:              20190201
    Website:                   http://climserv.ipsl.polytechnique.fr/cfmip-ob...
    References:                Chepfer et al, 2010, The GCM Oriented CALIPSO ...
    history:                   Wed Oct  9 15:42:51 2019: ncks -O -4 -L 1 MapL...
    NCO:                       netCDF Operators version 4.7.9 (Homepage = htt...
    nco_openmp_thread_number:  1

In [8]:
#f1 = xr.open_dataset('/home/rcostell/Observation_Files/NCEP_Reanalysis/Air_Temperature/Monthly/air.mon.mean.nc')
#f1.coords['lon'] = (f1.coords['lon'] + 180) % 360 - 180

#f1['time'] = xr.decode_cf(f1)['time']
#f1 = f1.resample(time='1M').mean()
#f1 = f1.isel(time=ind).air
#f1['time'] = f.time
#f = f.assign(air=f1)
#f1

In [9]:
f = f.sel(lat=slice(lat0, lat1)).sel(lon=slice(-15, 55))
cll_achv = f.cllcalipso.values
ice_achv = f.seaice_conc.values
lat = f.lat.values
lon = f.lon.values
time = f.time.values
nlat = len(lat)
nlon = len(lon)
ntime = len(time)

In [10]:
print("Start extracting")
print(cll_achv.shape)

def Binning_distance_from_ice_edge(ice_achv, var_achv, ntime, nlat, nlon, hem, outputcount = False, model = False):
    for j in range(ntime):
        if model:
            ice = ice_achv[j, :, :]
        else:
            ice = ice_achv[j, :, :]
            # This is the function that is used
        var = var_achv[j]
        #in this case cloud data is stored for each time j
        ndim = var.ndim
        #this is now 2-dimensional cloud data
        for i in range(nlon):
            tmp = ice[:,i]
            if ndim==3:
                var_tmp = var[:,:,i]
                nh = var_tmp.shape[0]
            else:
                var_tmp = var[:,i]
                #there are 2 dimensions so this function is used
                #Is there a reason that var should look different from var_tmp if both use the same longitudes?
            if hem == 'nh':
                ind1 = np.where(tmp <= 0.5)[0]
                #All sea ice values le
                ind2 = np.where(tmp > 0.5)[0]
                #Northern hemisphere is indeed selected
            else:
                ind1 = np.where(tmp > 0.5)[0]
                ind2 = np.where(tmp <= 0.5)[0]

            n1 = ind1.size
            n2 = ind2.size

            if i == 0:
                tmp0 = np.full((2*nlat), -999.)
                tmp0[nlat - n1: nlat] = tmp[ind1]
                tmp0[nlat: nlat + n2] = tmp[ind2]

                count0 = np.full((2*nlat), 1.)
                
                if ndim==3:
                    var_tmp0 = np.full((nh, 2*nlat), -999.)
                    var_tmp0[:,nlat - n1: nlat] = var_tmp[:,ind1]
                    var_tmp0[:,nlat: nlat + n2] = var_tmp[:,ind2]
                else:
                    var_tmp0 = np.full((2*nlat), -999.)
                    var_tmp0[nlat - n1: nlat] = var_tmp[ind1]
                    var_tmp0[nlat: nlat + n2] = var_tmp[ind2]
            else:
                tmp1 = np.full((2*nlat), -999.)
                tmp1[nlat - n1: nlat] = tmp[ind1]
                tmp1[nlat: nlat + n2] = tmp[ind2]

                count1 = np.full((2*nlat), 1.)
                
                if ndim==3:
                    var_tmp1 = np.full((nh, 2*nlat), -999.)
                    var_tmp1[:,nlat - n1: nlat] = var_tmp[:,ind1]
                    var_tmp1[:,nlat: nlat + n2] = var_tmp[:,ind2]
                else:
                    var_tmp1 = np.full((2*nlat), -999.)
                    var_tmp1[nlat - n1: nlat] = var_tmp[ind1]
                    var_tmp1[nlat: nlat + n2] = var_tmp[ind2]

                tmp0 = np.dstack((tmp0, tmp1))
                count0 = np.dstack((count0, count1))
                var_tmp0 = np.dstack((var_tmp0, var_tmp1))

        var_tmp0[var_tmp0 == -999.] = np.nan
        tmp0[tmp0 == -999.] = np.nan
        
        if ndim == 2:
            count0[np.isnan(var_tmp0)] = 0.
        
        count00 = nlon*np.nanmean(count0[0], axis = 1)
        tmp00 = np.nanmean(tmp0[0], axis = 1)
        if ndim == 3:
            var_tmp00 = np.nanmean(var_tmp0, axis = 2)
        else:
            var_tmp00 = np.nanmean(var_tmp0[0], axis = 1)

        if j == 0:
            count_out = count00
            var_out = var_tmp00
            ice_out = tmp00
        else:
            count_out = np.dstack((count_out, count00))        
            var_out = np.dstack((var_out, var_tmp00))
            ice_out = np.dstack((ice_out, tmp00))

    count_out = count_out[0]
    ice_out = ice_out[0]
    if ndim != 3:
        var_out = var_out[0]
    
    if ndim == 3:
        return var_out, ice_out
    else:
        if outputcount:
            return var_out, ice_out, count_out    
        else:
            return var_out, ice_out

Start extracting
(96, 30, 70)


In [11]:
cll_out, ice_out, count_out = Binning_distance_from_ice_edge(ice_achv, cll_achv, ntime, nlat, nlon, 'nh', outputcount = True)
ds_tmp1 = xr.DataArray(ice_out, coords=[np.arange(-nlat + 0.5, nlat + 0.5, 1.), f0.time.values],
                      dims=["distance","time"]).rename('seaice_conc')
ds_tmp10 = xr.DataArray(cll_out, coords=[np.arange(-nlat + 0.5, nlat + 0.5, 1.), f0.time.values],
                      dims=["distance","time"]).rename('cllcalipso')
ds_out = xr.merge([ds_tmp1, ds_tmp10])
print("done merging")
#ds_out.to_netcdf("/home/rcostell/Observation_Files/Binned/New_Observed_2D_CF.nc")
print('NH Job accomplished')

done merging
NH Job accomplished


/tmp/ipykernel_378138/2612452009.py:78: RuntimeWarning: Mean of empty slice
  tmp00 = np.nanmean(tmp0[0], axis = 1)
/tmp/ipykernel_378138/2612452009.py:82: RuntimeWarning: Mean of empty slice
  var_tmp00 = np.nanmean(var_tmp0[0], axis = 1)
